# Short Pulse Demonstration Notebook

## Aims of Notebook

- Mix and match long and very short pulses

## To Do

- Make notebook

In [322]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline
import numpy as np

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [323]:
# Pyro to communicate with board
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [4]:
# GEN_CH 1 is DAC_A and GEN_CH 0 is DAC_B
GEN_CH = 1
# RO_CH 1 is ADC_C and RO_CH 0 is ADC_D
RO_CH = 1

In [314]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, phase=phase, gain=gain)

        # idata, qdata = [], []
        # for i in range(96):
        #     x_vals = np.linspace(0, 2 * np.pi, 100)
        #     sinusoid = 32767 * np.sin(x_vals)
        #     idata.extend(sinusoid)
        #     qdata.extend(sinusoid)
        idata0 = np.full(shape=9600, fill_value=16000, dtype=int)
        qdata0 = np.full(shape=9600, fill_value=16000, dtype=int)

        for i in range(1000, 1050):
            idata0[i] = 0
            qdata0[i] = 0
        
        idata1 = np.full(shape=9600, fill_value=24000, dtype=int)
        qdata1 = np.full(shape=9600, fill_value=24000, dtype=int)
        
        self.add_envelope(ch=res_ch, name="measure", idata=idata0, qdata=qdata0)
        self.add_envelope(ch=res_ch, name="test", idata=idata1, qdata=qdata1)
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        freq = self.freq2reg(self.cfg["pulse_freq"],gen_ch=self.cfg["res_ch"], ro_ch=self.cfg["ro_chs"][0])

        self.set_pulse_registers(ch=self.cfg["res_ch"], style="arb", waveform="measure", freq=freq)
        self.pulse(ch=self.cfg["res_ch"])

        self.set_pulse_registers(ch=self.cfg["res_ch"], style="const", length=100, freq=freq)
        self.pulse(ch=self.cfg["res_ch"])

        self.set_pulse_registers(ch=self.cfg["res_ch"], style="arb", waveform="test", freq=freq)
        self.pulse(ch=self.cfg["res_ch"])

        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        # self.measure(pulse_ch=self.cfg["res_ch"], 
        #              adcs=self.ro_chs,
        #              pins=[0], 
        #              adc_trig_offset=self.cfg["adc_trig_offset"],
        #              wait=True,
        #              syncdelay=self.us2cycles(self.cfg["relax_delay"]))

        
        # equivalent to the following:
#         self.trigger(adcs=self.ro_chs,
#                      pins=[0], 
#                      adc_trig_offset=self.cfg["adc_trig_offset"])
#         self.pulse(ch=self.cfg["res_ch"])
#         self.wait_all()
#         self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

In [312]:
config={"res_ch":GEN_CH, # --Fixed
        "ro_chs":[RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "arb", # --Fixed
        
        "length":100, # [Clock ticks]
        # Try varying length from 10-100 clock ticks

        "sigma": 30,
        
        "readout_length":100, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":30000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 50, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":1
        # Try varying soft_avgs from 1 to 200 averages

       }

In [320]:
prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, progress=False)

ValueError: cannot reshape array of size 0 into shape (0,newaxis,2)

In [14]:
# from scipy.signal import welch
# complex_iq = iq.dot([1,1j])

# # fs_adc = soc['readouts'][RO_CH]['fs']
# fs_adc = 4423.68

# fft_freqs, psd = welch(iq[:,0], fs_adc, return_onesided=True, detrend=False, nperseg=512)
# plt.semilogy(fft_freqs, psd)
# plt.xlabel("absolute frequency [MHz]")
# plt.ylabel("power [a.u.]");